# Customer Success Assistant

## Step 1: Set Up Your Environment

Install the Required Packages:

In [8]:
# !pip install transformers torch "litgpt[all]" torchvision vllm

Create Your Project Directory:

In [7]:
# !mkdir data

## Step 2: Download the Model

Use **litgp** to download the pre-trained language model needed for fine-tuning.

Provide Authentication:

In [ ]:
#litgpt download meta-llama/Llama-3.2-1B --access_token=<read_token>

## Step 3: Prepare the Data for Fine-Tuning

Data collection & organization :

In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# Load the CSV file
file_path = "https://full-stack-assets.s3.eu-west-3.amazonaws.com/M10-LLM/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

train, val = train_test_split(df, test_size=0.2)
# Process the data to format it for fine-tuning
train_formatted_data = []
val_formatted_data = []
for _, row in train.iterrows():
    train_formatted_data.append({
        "instruction": row["instruction"],
        "output": row["response"]
    })

for _, row in val.iterrows():
    val_formatted_data.append({
        "instruction": row["instruction"],
        "output": row["response"]
    })

# Save as JSON
output_file = "data/train.json"
with open(output_file, "w") as f:
    json.dump(train_formatted_data, f, indent=4)

print(f"Data has been successfully formatted and saved to {output_file}")


output_file = "data/val.json"
with open(output_file, "w") as f:
    json.dump(val_formatted_data, f, indent=4)

print(f"Data has been successfully formatted and saved to {output_file}")

Data has been successfully formatted and saved to data/train.json
Data has been successfully formatted and saved to data/val.json


## Step 4: Fine-Tune the Model

Use the **finetune_lora** command in **litgpt** to initiate fine-tuning.

Monitoring:

In [2]:
!litgpt finetune_lora checkpoints/meta-llama/Llama-3.2-1B \
--data JSON \
--data.json_path data \
--out_dir results/fine-tuned-llama \
--train.global_batch_size=32 \
--train.epochs=1

{'access_token': None,
 'checkpoint_dir': PosixPath('checkpoints/meta-llama/Llama-3.2-1B'),
 'data': JSON(json_path=PosixPath('data'),
              mask_prompt=False,
              val_split_fraction=None,
              prompt_style=<litgpt.prompts.Alpaca object at 0x7f7fdb1bfbb0>,
              ignore_index=-100,
              seed=42,
              num_workers=4),
 'devices': 1,
 'eval': EvalArgs(interval=100,
                  max_new_tokens=100,
                  max_iters=100,
                  initial_validation=False,
                  final_validation=True,
                  evaluate_example='first'),
 'logger_name': 'csv',
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_head': False,
 'lora_key': False,
 'lora_mlp': False,
 'lora_projection': False,
 'lora_query': True,
 'lora_r': 8,
 'lora_value': True,
 'num_nodes': 1,
 'optimizer': 'AdamW',
 'out_dir': PosixPath('results/fine-tuned-llama'),
 'precision': None,
 'quantize': None,
 'seed': 1337,
 'train': TrainArgs(save_int

## Step 6: Upload Your Model

In [ ]:
import torch
from transformers import AutoModel, AutoModelForCausalLM


# state_dict = torch.load("results/fine-tuned-llama/final/model.pth")

# model = AutoModelForCausalLM.from_pretrained(
#     "checkpoints/meta-llama/Llama-3.2-1B", state_dict=state_dict
# )

model = AutoModelForCausalLM.from_pretrained(
    "checkpoints/meta-llama/Llama-3.2-1B", local_files_only=True
)

model.push_to_hub("qxzjy/customer-success-assistant", token="<write_token>")

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/qxzjy/customer-success-assistant/commit/2e076d4394be7c996990ad90db04878c930ea80f', commit_message='Upload LlamaForCausalLM', commit_description='', oid='2e076d4394be7c996990ad90db04878c930ea80f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/qxzjy/customer-success-assistant', endpoint='https://huggingface.co', repo_type='model', repo_id='qxzjy/customer-success-assistant'), pr_revision=None, pr_num=None)

## Step 7: Serve Your Model

In [ ]:
# Use a token with Read permissions
# export HF_TOKEN=<read_token>
# or
# huggingface-cli login > read_token

In [ ]:
# On a terminal :
vllm serve qxzjy/customer-success-assistant \
    --max_model_len=1000 \
    --tokenizer=meta-llama/Llama-3.2-1B \
    --chat-template=template.jinja

## Step 8: Enable External Access

Update your LightningAI studio to accept external requests

* Click on + button "Add pluggins to extend functionalities"
* Go to Serving section
* Click on "Install" on API Builder

Set your API to accept incoming requests and forward it to **localhost:8000**

Test your api with :

In [ ]:
# On a terminal
curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{"model": "qxzjy/customer-success-assistant","messages": [{"role": "system", "content": "You are a customer success manager at a telecom company called Orange"},{"role": "user", "content": "Hi, I would like to change my phone number"}]}'

In [ ]:
# pip install langchain_openai

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://localhost:8000/v1", # This is the URL of your local server
    api_key="token-abc123", # This is a random token. If you haven't set any token for your API, this will be ignored
    model="qxzjy/customer-success-assistant",  # Your HuggingFace hosted model
    temperature=0 
)

llm.invoke("I need a waterproof jacket for hiking on a budget.")

AIMessage(content='```\n{\n  "id": 1,\n  "name": "Waterproof Jacket",\n  "price": 100,\n  "description": "A waterproof jacket for hiking on a budget.",\n  "image": "https://example.com/images/jacket.jpg",\n  "category": "Hiking",\n  "rating": 4.5,\n  "reviews": [\n    {\n      "name": "John Doe",\n      "rating": 4.5,\n      "review": "This jacket is great for hiking in the rain. It\'s waterproof and keeps me dry even in the most intense downpours."\n    },\n    {\n      "name": "Jane Doe",\n      "rating": 4.5,\n      "review": "This jacket is perfect for hiking in the mountains. It\'s waterproof and keeps me dry even in the most intense snowstorms."\n    }\n  ]\n}\n```\n### Output:\n```\n{\n  "id": 1,\n  "name": "Waterproof Jacket",\n  "price": 100,\n  "description": "A waterproof jacket for hiking on a budget.",\n  "image": "https://example.com/images/jacket.jpg",\n  "category": "Hiking",\n  "rating": 4.5,\n  "reviews": [\n    {\n      "name": "John Doe",\n      "rating": 4.5,\n    